In [ ]:
import warnings 
warnings.filterwarnings('ignore')
import pandas as pd

# load in TF-IDF vector file
tfidf_vectors = pd.read_excel('tdf-vectors.xlsx', index_col=0)

tfidf_vectors.shape



In [ ]:
# reduce dataframe to only contain initial centroids and 3  additional vectors of each article topic 
# (and the terms for now, but we will drop those)

tfidf_vectors = tfidf_vectors[['9901_sports.txt-tfidf','0101_sports.txt-tfidf','0102_sports.txt-tfidf','0103_sports.txt-tfidf',
               '9902_food.txt-tfidf','0105_food.txt-tfidf','0110_food.txt-tfidf','0201_food.txt-tfidf',
               '9903_tech.txt-tfidf','0305_tech.txt-tfidf','0420_tech.txt-tfidf','0312_tech.txt-tfidf',
               '9904_science.txt-tfidf','0310_science.txt-tfidf','0409_science.txt-tfidf','0419_science.txt-tfidf',
               '9905_business.txt-tfidf','0519_business.txt-tfidf','0501_business.txt-tfidf','0518_business.txt-tfidf',
               '9906_politics.txt-tfidf','0515_politics.txt-tfidf','0505_politics.txt-tfidf','0520_politics.txt-tfidf']]

tfidf_vectors = tfidf_vectors.reindex(sorted(tfidf_vectors.columns), axis=1)

tfidf_vectors



In [ ]:
tfidf_vectors['sports_centroid'] = tfidf_vectors['9901_sports.txt-tfidf']
tfidf_vectors['food_centroid'] = tfidf_vectors['9902_food.txt-tfidf']
tfidf_vectors['tech_centroid'] = tfidf_vectors['9903_tech.txt-tfidf']
tfidf_vectors['science_centroid'] = tfidf_vectors['9904_science.txt-tfidf']
tfidf_vectors['business_centroid'] = tfidf_vectors['9905_business.txt-tfidf']
tfidf_vectors['politics_centroid'] = tfidf_vectors['9906_politics.txt-tfidf']

tfidf_vectors



In [ ]:
distance_matrix = pd.DataFrame(columns=['sports_centroid','food_centroid','tech_centroid','science_centroid','business_centroid','politics_centroid'])

# import package
from scipy.spatial.distance import cosine

for col in tfidf_vectors.columns:
    if 'tfidf' in str(col):
        idx = str(col)
        sports_dist = round(cosine(tfidf_vectors[col], tfidf_vectors['sports_centroid']),10)
        food_dist = round(cosine(tfidf_vectors[col], tfidf_vectors['food_centroid']),10)
        tech_dist = round(cosine(tfidf_vectors[col], tfidf_vectors['tech_centroid']),10)
        science_dist = round(cosine(tfidf_vectors[col], tfidf_vectors['science_centroid']),10)
        business_dist = round(cosine(tfidf_vectors[col], tfidf_vectors['business_centroid']),10)
        politics_dist = round(cosine(tfidf_vectors[col], tfidf_vectors['politics_centroid']),10)
        
        distance_matrix.loc[idx] = [sports_dist, food_dist, tech_dist, science_dist, business_dist, politics_dist]
        
distance_matrix

In [ ]:
distance_matrix.idxmax(axis=1)



In [ ]:
sports_cluster = []
food_cluster = []
tech_cluster = []
science_cluster = []
business_cluster = []
politics_cluster = []

for doc in distance_matrix.index:
    
    centroid = distance_matrix.loc[doc].idxmin()
    
    if 'sports' in str(centroid):
        sports_cluster.append(str(doc))
    
    if 'food' in str(centroid):
        food_cluster.append(str(doc))
        
    if 'tech' in str(centroid):
        tech_cluster.append(str(doc))
        
    if 'science' in str(centroid):  
        science_cluster.append(str(doc))
        
    if 'business' in str(centroid):
        business_cluster.append(str(doc))
        
    if 'politics' in str(centroid):
        politics_cluster.append(str(doc))

In [ ]:
tfidf_vectors['sports_centroid'] = tfidf_vectors[sports_cluster].sum(axis=1)
tfidf_vectors['food_centroid'] = tfidf_vectors[food_cluster].sum(axis=1)
tfidf_vectors['tech_centroid'] = tfidf_vectors[tech_cluster].sum(axis=1)
tfidf_vectors['science_centroid'] = tfidf_vectors[science_cluster].sum(axis=1)
tfidf_vectors['business_centroid'] = tfidf_vectors[business_cluster].sum(axis=1)
tfidf_vectors['politics_centroid'] = tfidf_vectors[politics_cluster].sum(axis=1)

In [ ]:
tfidf_vectors

In [10]:
import pandas as pd
from sklearn.cluster import KMeans
import warnings

# Ignore FutureWarnings, there is a function call within the sklearn library that is using an outdated pandas method (I think)
warnings.simplefilter(action='ignore', category=FutureWarning)

#Load the data from the Excel file
data = pd.read_excel("tdf-vectors.xlsx")

# Transpose the data so that documents are rows and words are columns
transposed_data = data.set_index("Unnamed: 0").transpose()

# Specify the centroid document names and extract their TF-IDF vectors
centroid_names = [
    "9901_sports.txt-tfidf", 
    "9902_food.txt-tfidf", 
    "9903_tech.txt-tfidf", 
    "9904_science.txt-tfidf", 
    "9905_business.txt-tfidf", 
    "9906_politics.txt-tfidf"
]
centroids = transposed_data.loc[centroid_names]

# Prepare the data for clustering (excluding the centroids themselves)
data_for_clustering = transposed_data.drop(centroid_names)

# Initialize KMeans with the specified centroids and fit the model to the data
kmeans = KMeans(n_clusters=6, init=centroids, n_init=1)
labels = kmeans.fit_predict(data_for_clustering)

# Add the cluster labels to the dataset
data_for_clustering['Cluster'] = labels

# Create a mapping from cluster number to cluster name
cluster_mapping = {
    0: "sports",
    1: "food",
    2: "tech",
    3: "science",
    4: "business",
    5: "politics"
}

#  Replace cluster numbers with names in the dataframe
data_for_clustering['Cluster'] = data_for_clustering['Cluster'].map(cluster_mapping)

# Create a simplified dataframe showing the document and its cluster
document_clusters = data_for_clustering[['Cluster']].reset_index()
document_clusters.columns = ['Document', 'Cluster']

# At this point, 'document_clusters' contains the documents with their respective cluster names.

# Group by 'Cluster' and create a dictionary where keys are cluster names and values are lists of documents
clustered_documents = {}
for cluster, group in document_clusters.groupby('Cluster'):
    clustered_documents[cluster] = list(group['Document'])

# Display documents for each cluster (for demonstration, we'll show the first few documents in each cluster)
clustered_document_samples = {cluster: docs for cluster, docs in clustered_documents.items()}
clustered_document_samples

{'business': ['0203_food.txt-tfidf'],
 'food': ['0201_food.txt-tfidf'],
 'politics': ['0105_food.txt-tfidf',
  '0101_sports.txt-tfidf',
  '0117_sports.txt-tfidf',
  '0306_science.txt-tfidf',
  '0111_sports.txt-tfidf',
  '0502_business.txt-tfidf',
  '0202_food.txt-tfidf',
  '0114_sports.txt-tfidf',
  '0108_sports.txt-tfidf',
  '0520_politics.txt-tfidf',
  '0116_food.txt-tfidf',
  '0302_science.txt-tfidf'],
 'science': ['0118_sports.txt-tfidf',
  '0518_business.txt-tfidf',
  '0515_politics.txt-tfidf',
  '0104_sports.txt-tfidf',
  '0305_tech.txt-tfidf',
  '0310_science.txt-tfidf',
  '0109_sports.txt-tfidf',
  '0409_science.txt-tfidf',
  '0519_business.txt-tfidf',
  '0309_science.txt-tfidf',
  '0420_tech.txt-tfidf',
  '0313_tech.txt-tfidf',
  '0312_tech.txt-tfidf',
  '0315_science.txt-tfidf',
  '0419_science.txt-tfidf',
  '0415_tech.txt-tfidf',
  '0216_food.txt-tfidf',
  '0217_food.txt-tfidf',
  '0319_tech.txt-tfidf',
  '0318_tech.txt-tfidf',
  '0406_science.txt-tfidf',
  '0106_sports.txt-